In [1]:
from bs4 import BeautifulSoup as bs
import requests,re,codecs
import pandas as pd
import csv
textParsed = ""

In [2]:
lines = '''pageInfo:bksMain
login_chk:null
LOGIN_SN:null
LOGIN_NAME:null
indexName:news
keyword:이랜드 뉴코아 공동투쟁본부
byLine:
searchScope:1
searchFtr:1
startDate:2006-01-01
endDate:2008-12-31
sortMethod:date
contentLength:100
providerCode:
categoryCode:
incidentCode:
dateCode:
highlighting:true
sessionUSID:
sessionUUID:test
listMode:
categoryTab:
newsId:
delnewsId:
delquotationId:
delquotationtxt:
filterProviderCode:
filterCategoryCode:
filterIncidentCode:
filterDateCode:
filterAnalysisCode:
startNo:1
resultNumber:100
topmenuoff:
resultState:
keywordJson:{"searchDetailTxt1":"이랜드 뉴코아 공동투쟁본부","agreeDetailTxt1":"","needDetailTxt1":"","exceptDetailTxt1":"","o_id":"option1","startDate":"2006-01-01","endDate":"2008-12-31","providerNm":"","categoryNm":"","incidentCategoryNm":"","providerCode":"","categoryCode":"","incidentCategoryCode":"","searchFtr":"1","searchScope":"1","searchKeyword":"이랜드 뉴코아 공동투쟁본부"}
keywordFilterJson:
realKeyword:
keywordYn:Y
totalCount:
interval:
quotationKeyword1:
quotationKeyword2:
quotationKeyword3:
printingPage:
searchFromUseYN:N
searchFormName:
searchFormSaveSn:
mainTodayPersonYn:
period:'''.splitlines()

In [3]:
data = {}
for line in lines:
    key,value = line.split(':',1)
    if value=='null':
        value = None
    data[key] = value

In [4]:
result_url = "https://www.bigkinds.or.kr/news/detailSearch.do"
url_list = {}
for index in range(1,2):
    data['startNo'] = index
    response = requests.post(result_url,data=data)
    html = response.text
    soup = bs(html,"html.parser")
    for tag in soup.select('.resultList li h3'):
        key = tag.text.strip()
        doc_id = tag['id'].replace('news_','')
        value = 'https://www.bigkinds.or.kr/news/detailView.do?docId={}&returnCnt=1'.format(doc_id)
        url_list[key] = value
main_df = pd.DataFrame(list(url_list.items()),columns=['Title','url'])

In [5]:
main_df.head()

,Title,url
0,"뉴코아노조 ""사측, 노조양보안 수용해야""",https://www.bigkinds.or.kr/news/detailView.do?...
1,이랜드 1인시위자-점주 충돌 /폭행사고 났는데 경찰은 ‘뒷짐’,https://www.bigkinds.or.kr/news/detailView.do?...
2,"[단독] 경찰, 이랜드 노조원 이례적 ‘지문채취 영장’ 파문",https://www.bigkinds.or.kr/news/detailView.do?...
3,"경찰, ''이랜드 농성'' 주도 3명 구속… 노조원 등 192명 불구속입건",https://www.bigkinds.or.kr/news/detailView.do?...
4,"수원지법, 뉴코아집행부 3명 영장발부",https://www.bigkinds.or.kr/news/detailView.do?...


In [6]:
main_df.to_excel('공동투_url.xlsx')

In [7]:
date_dict = {}
articles = {}
provider_dict = {}

for url in main_df['url']:
    try:
        response = requests.post(url)
        html = response.text
        dict1 = eval(html)
        content = dict1['detail']['CONTENT']
        articles[url] = content
        date=dict1['detail']['DATE']
        date_dict[url]=date
        provider=dict1['detail']['PROVIDER']
        provider_dict[url]=provider
    except:
        pass

    
articles_df = pd.DataFrame(list(articles.items()),columns=['url','Contents'])
total_df = pd.merge(main_df,articles_df,on="url")
date_df = pd.DataFrame(list(date_dict.items()),columns=['url','Date'])
total_df = pd.merge(total_df,date_df,on='url')
provider_df = pd.DataFrame(list(provider_dict.items()),columns=['url','Provider'])
total_df = pd.merge(total_df,provider_df,on='url')

In [8]:
print(total_df['Provider'].unique())

['중부일보' '한겨레' '서울신문' '세계일보' '국민일보' '국제신문' '충청투데이' '한국경제' '부산일보' '머니투데이'
 '서울경제']


In [9]:
total_df.head()

,Title,url,Contents,Date,Provider
0,"뉴코아노조 ""사측, 노조양보안 수용해야""",https://www.bigkinds.or.kr/news/detailView.do?...,뉴코아-이랜드 일반노동조합 공동투쟁본부는 28일 서울 서초구 뉴코아 강남점 앞에서 ...,20071228,중부일보
1,이랜드 1인시위자-점주 충돌 /폭행사고 났는데 경찰은 ‘뒷짐’,https://www.bigkinds.or.kr/news/detailView.do?...,이랜드그룹의 비정규직 노동자 해고에 항의하며 매장 앞에서 1인시위를 하려던 대학생들...,20070806,한겨레
2,"[단독] 경찰, 이랜드 노조원 이례적 ‘지문채취 영장’ 파문",https://www.bigkinds.or.kr/news/detailView.do?...,경찰이 지난달 31일 연행한 이랜드 노조원 197명을 소환·조사하면서 노조원 가운데...,20070803,서울신문
3,"경찰, ''이랜드 농성'' 주도 3명 구속… 노조원 등 192명 불구속입건",https://www.bigkinds.or.kr/news/detailView.do?...,경찰은 지난달 29∼31일 서울 뉴코아 강남점을 재점거해 농성을 벌인 혐의(업무방해...,20070803,세계일보
4,"수원지법, 뉴코아집행부 3명 영장발부",https://www.bigkinds.or.kr/news/detailView.do?...,이랜드 매장 점거 농성 등 시위를 벌이고 있는 뉴코아 노조 집행부 3명에 대해 경찰...,20070802,중부일보


In [11]:
total_df.to_excel("News_공동투+이랜드+뉴코아.xlsx")